## Regression with Keras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from scipy.stats.mstats import winsorize

In [2]:
df=pd.read_csv('C:\\Users\\Goksun\\Downloads\\appliances-energy.csv')
df.head(3)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668


In [3]:
#Due to lot of zero enteries, this column is of not much use and will be ignored in rest of the model.
df.drop(columns='lights',axis=1,inplace=True)

In [4]:
df['date'] = pd.to_datetime(df['date'])
df= df.set_index('date')
df.index.name ='time'
df['date'] = df.index.date
df['day'] = df.index.day
df['month'] = df.index.month
df['hour'] = df.index.hour

In [5]:
df2=pd.concat([df.groupby(['month','day','hour']).Appliances.sum(),
           df.groupby(['month','day','hour']).mean().drop('Appliances',axis=1)],axis=1)

In [6]:
def f(x):
    x = x +273.15
    return x

In [7]:
dfk=df2.copy()
temp_list=['T1','T2','T3','T4','T6','T5','T7','T8','T9','T_out','Tdewpoint']
for i in temp_list:
    dfk[i] = dfk[i].apply(f)

In [8]:
dfkw=pd.DataFrame()
others=['RH_4','RH_6','T7']
for i in others:
    dfkw['w_%s'%i]=dfk[i]

In [9]:
#One by one, we kept winsorization low ratio because it might cause some side effects if we increase that ratio too much.
dfkw['w_Appliances']=winsorize(dfk.Appliances,(0,0.105))
dfkw['w_T1']=winsorize(dfk.T1,(0.05,0.05))
dfkw['w_RH_1']=winsorize(dfk.RH_1,(0.01,0.01))
dfkw['w_T2']=winsorize(dfk.T2,(0,0.04))
dfkw['w_RH_2']=winsorize(dfk.RH_2,(0.01,0.01))
dfkw['w_T3']=winsorize(dfk.T3,(0,0.02))
dfkw['w_T4']=winsorize(dfk.T4,(0.01,0.01))
dfkw['w_T5']=winsorize(dfk.T5,(0,0.01))
dfkw['w_RH_5']=winsorize(dfk.RH_5,(0.001,0.07))
dfkw['w_T6']=winsorize(dfk.T6,(0,0.05))
dfkw['w_RH_7']=winsorize(dfk.RH_7,(0,0.04))
dfkw['w_T_out']=winsorize(dfk.T_out,(0,0.04))
dfkw['w_Press_mm_hg']=winsorize(dfk.Press_mm_hg,(0.012,0))
dfkw['w_RH_out']=winsorize(dfk.RH_out,(0.02,0))
dfkw['w_Windspeed']=winsorize(dfk.Windspeed,(0,0.02))
dfkw['w_Visibility']=winsorize(dfk.Visibility,(0.05,0.125))
dfkw['w_RH_3']=winsorize(dfk.RH_3,(0,0.001))
dfkw['w_T8']=winsorize(dfk.T8,(0.05,0))
dfkw['w_RH_8']=winsorize(dfk.RH_8,(0,0.01))
dfkw['w_T9']=winsorize(dfk.T9,(0,0.001))
dfkw['w_RH_9']=winsorize(dfk.RH_9,(0.001,0.001))
dfkw['w_Tdewpoint']=winsorize(dfk.Tdewpoint,(0,0.001))
dfkw['w_rv1']=winsorize(dfk.rv1,(0.005,0.005))
dfkw['w_rv2']=winsorize(dfk.rv2,(0.005,0.005))

In [10]:
df3=dfkw.copy()
dflog=pd.DataFrame()
for col in df3.columns:
    dflog['log_%s'%col]=np.log(df3[col])

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(dflog)
min_max=scaler.transform(dflog)
scale_df=pd.DataFrame(min_max)
scale_df=scale_df.rename({0:'Appliances',1:'T1', 2:'RH_1', 3:'T2', 4:'RH_2', 5:'T3', 6:'RH_3', 7:'T4', 8:'RH_4',
       9:'T5', 10:'RH_5', 11:'T6', 12:'RH_6', 13:'T7', 14:'RH_7', 15:'T8', 16:'RH_8', 17:'T9', 18:'RH_9',
       19:'T_out', 20:'Press_mm_hg', 21:'RH_out', 22:'Windspeed', 24:'Visibility',25:'Tdewpoint', 26:'rv1', 27:'rv2'},axis=1)

In [12]:
#Train-test split
from sklearn.model_selection import train_test_split
X = scale_df.drop('Appliances',axis=1).values
y = scale_df.Appliances.values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 123)

In [13]:
import warnings
warnings.filterwarnings('ignore')
from keras.layers import Dense
from keras.models import Sequential
model=Sequential()

Using TensorFlow backend.


In [14]:
girdi=X_train.shape[1]
model.add(Dense(100,activation='relu',input_shape=(girdi,),name='GizliK1'))
model.add(Dense(50,activation='relu',name='GizliK2'))
model.add(Dense(25,activation='relu',name='GizliK3'))
model.add(Dense(1,name='Cikti'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
GizliK1 (Dense)              (None, 100)               2700      
_________________________________________________________________
GizliK2 (Dense)              (None, 50)                5050      
_________________________________________________________________
GizliK3 (Dense)              (None, 25)                1275      
_________________________________________________________________
Cikti (Dense)                (None, 1)                 26        
Total params: 9,051
Trainable params: 9,051
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
model.fit(X_train,y_train,epochs=50,verbose=0,batch_size=32)

In [17]:
train=model.evaluate(X_train,y_train)[1]
test=model.evaluate(X_test,y_test)[1]
print('MSE value of Train Set:',train)
print('MSE value of Test Set:',test)

658/658 [==============================] - 0s 29us/step
MSE value of Train Set: 0.0011687694350257516
MSE value of Test Set: 0.0015866541070863605


In [18]:
print('RMSE value for Train Set:',np.sqrt(train))
print('RMSE value for Test Set:',np.sqrt(test))

RMSE value for Train Set: 0.03418727007272958
RMSE value for Test Set: 0.03983282700344479
